Урок 2. Парсинг HTML

In [14]:
import requests
from bs4 import BeautifulSoup as bs   # у нас html странички, потому берем это     
from pprint import pprint
import time
import pandas as pd

# по РОСКОНТРОЛЮ------------------------------------------------------------------------
# Необходимо собрать информацию по продуктам питания с сайтов: 
# https://rskrf.ru/ratings/produkty-pitaniya/ и https://roscontrol.com/category/produkti/ 
#Получившийся список должен содержать:
#        *Наименование продукта
#        *Категорию продукта        (например Бакалея)
#        *Подкатегорию продукта    (апример "Рис круглозерный")
#        *Параметр "безопасность"
#        *Параметр "качество"
#        *Общий балл
#        *Сайт, откуда получена информация
# Данная структура должна быть одинаковая для продуктов с обоих сайтов. 
# Общий результат можно вывести с помощью dataFrame через pandas.

head ={'User-Agent': 'YaBrowser/19.7.5.90.01'}
main_link = 'https://roscontrol.com/category/produkti/'
site_name = 'https://roscontrol.com/'
forbidden_page = 'https://roscontrol.com/product/produkti/'
N,NN = 100,200
lst =[]
# по категории [1]:
time.sleep(0.2); req2 = requests.get(main_link).text
parsed_req2 = bs(req2,'html.parser')
block_2 = parsed_req2.find('div',{'class':'grid-row grid-flex-mobile'})
list_categories = block_2.findAll('a')
for en_categories in list_categories:
    categorie_name = en_categories['href'].split('/')[3]
    categorie_link = main_link + categorie_name + '/'
    
    time.sleep(0.2); req2a = requests.get(categorie_link).text
    parsed_req2a = bs(req2a,'html.parser')
    block_2a = parsed_req2a.find('div',{'class':'grid-row grid-flex-mobile'})
    if(block_2a==None): break # если список закончился, а теги еще остались...
    list_goods = block_2a.findAll('a')
    for en_goods in list_goods:
        good_name = en_goods['href'].split('/')[4]
        good_link = categorie_link + good_name + '/'
        
        time.sleep(0.2); req2b = requests.get(good_link).text
        parsed_req2b = bs(req2b,'html.parser')
        block_2b = parsed_req2b.find('div',{'class':'AJAX_root'})
        if(block_2b==None): break # если список закончился, а теги еще остались...
        list_points = block_2b.findAll('a')
        
        for en_points in list_points:
            point_name = en_points['href'].split('/')[2]
            point_link = 'https://roscontrol.com/product/' + point_name + '/'
            print(point_link)
            if point_link == forbidden_page: continue                # пустая карточка продукта
            time.sleep(0.2); req2c = requests.get(point_link).text
            parsed_req2c = bs(req2c,'html.parser')
            dngr_rate,qual_rate,total_rate = '*','*','*'

#            total_block  = parsed_req2c.find('div',{'id':'product__single-ref-total'})
#            if(total_block==None): break # если список закончился, а теги еще остались...
#            total_rate   = total_block.find('div',{'class':'total green'}).text
#            print(point_name, total_rate)

            q_block = parsed_req2c.find('div',{'id':'product__single-rev-rating'})
            if (q_block==None): continue         # страница не найдена!!
            q_total = q_block.find('div',{'id':'product__single-rev-total'})
            green   = q_total.find('div',{'class':'total green'})
            if(not(green==None)):  total_rate = green.string
#
            q_rates = q_block.find('div',{'id':'product__single-rev-rate'})
#
            grp_all  = q_rates.findAll('div',{'class':'rate-item group'})
            if(len(grp_all)>0):
                dngr = grp_all[0].find('div',{'class':'rate-item__value'})
                dngr_tag = dngr.find('span')
                dngr_rate = dngr_tag.string
    #
                if(len(grp_all)>3):
                    qual = grp_all[3].find('div',{'class':'rate-item__value'})
                    qual_tag = qual.find('span')
                    qual_rate = qual_tag.string

            
            lst.append([point_name,categorie_name,good_name,dngr_rate,qual_rate,total_rate,site_name])
#            print(len(lst))
            if(len(lst)>N): break
        if(len(lst)>N): break
    if(len(lst)>N): break
#======================================================================================================
main_link = 'https://rskrf.ru/ratings/produkty-pitaniya/'
site_name = 'https://rskrf.ru/'
goods_link ='https://rskrf.ru/goods/'
print('start parsing the: ',site_name)

# по категории [1]:
time.sleep(0.2); req1 = requests.get(main_link).text
parsed_req1 = bs(req1,'html.parser')
block_1 = parsed_req1.find('div',{'class':'categories'})
categories_list = block_1.findChildren(recursive=False)
for en in categories_list:
    en_link = en.find('a')['href']
    categorie_name = en_link.split('/')[3]
    categorie_link = main_link + categorie_name + '/'
#    print('*** ',categorie_name,categorie_link)
    
    # по sub-категории [2]:
    time.sleep(0.2);  req1_i = requests.get(categorie_link).text
    parsed_req_i = bs(req1_i,'html.parser')
    block_1i = parsed_req_i.find('div',{'class':'categories'})
    items_list = block_1i.findChildren(recursive=False)
    for it in items_list:
        it_link = it.find('a')['href']
        item_name = it_link.split('/')[4]
        item_link = categorie_link + item_name + '/'
        print('****** ',item_link)

        # по бренду [0]:
#        print('must be: https://rskrf.ru/ratings/produkty-pitaniya/bakaleya/sukhie-zavtraki/')
        time.sleep(0.2); req1_ij = requests.get(item_link).text
        parsed_req_brand = bs(req1_ij,'html.parser')
        block_goods = parsed_req_brand.find('div',{'class':'product-row row rating-id'})
        list_prod = block_goods.findAll('div',{'class':'col-xl-4 col-lg-4 col-md-4 product'})        
        card = list_prod[0].find('div',{'class':'card'}) 
#        print(type(card))
#        link_goods = card.findAll('a')['href']    
#        print(len(link_goods))
#        print(link_goods[0])
        
        lst.append(['*',categorie_name,item_name,'*','*','*',site_name])

            
#        lst.append([point_name,categorie_name,good_name,dngr_rate,qual_rate,total_rate,site_name])
#        if(len(lst)>NN): break
        if(len(lst)>NN): break
    if(len(lst)>NN): break


#======================================================================================================

DF = pd.DataFrame(lst,columns=['point','category','sub_ctgr','dangrless','quality','rating','site'])
DF
#categories_lis_categories

https://roscontrol.com/product/moloko-ekomilk/
https://roscontrol.com/product/krasnaya-tsena-32-ultrapasterizovannoe/
https://roscontrol.com/product/moloko-prostokvashino-3-2/
https://roscontrol.com/product/selo-zelenoe-32-ultrapasterizovannoe/
https://roscontrol.com/product/avida-32-pasterizovannoe/
https://roscontrol.com/product/prostokvashino-otbornoe-ot-34-do-45-pasterizovannoe/
https://roscontrol.com/product/ostankinskoe-32-ultrapasterizovannoe/
https://roscontrol.com/product/sarafanovo-32-ultrapasterizovannoe/
https://roscontrol.com/product/valio-32-ultrapasterizovannoe/
https://roscontrol.com/product/avida-32-ultrapasterizovannoe/
https://roscontrol.com/product/vkusnotieievo-3-2-pastierizovannoie/
https://roscontrol.com/product/asenevskaya-ferma-32-pasterizovannoe/
https://roscontrol.com/product/ruzskoe-32-4-pasterizovannoe-tselnoe/
https://roscontrol.com/product/domik-v-derevne-35-tselnoe-pasterizovannoe/
https://roscontrol.com/product/mamulya-32-ultrapasterizovannoe/
https://r

******  https://rskrf.ru/ratings/produkty-pitaniya/maslo-rastitelnoe/rafinirovannoe-podsolnechnoe-maslo/
******  https://rskrf.ru/ratings/produkty-pitaniya/molochnye-produkty/yogurty/
******  https://rskrf.ru/ratings/produkty-pitaniya/molochnye-produkty/detskie-molochnye-smesi/
******  https://rskrf.ru/ratings/produkty-pitaniya/molochnye-produkty/syrok-glazirovannyy-vanilnyy/
******  https://rskrf.ru/ratings/produkty-pitaniya/molochnye-produkty/syr-plavlenyy-slivochnyy/
******  https://rskrf.ru/ratings/produkty-pitaniya/molochnye-produkty/kefir/
******  https://rskrf.ru/ratings/produkty-pitaniya/molochnye-produkty/moloko/
******  https://rskrf.ru/ratings/produkty-pitaniya/molochnye-produkty/morozhenoe/
******  https://rskrf.ru/ratings/produkty-pitaniya/molochnye-produkty/slivochnoe-maslo-72-5-zhirnosti/
******  https://rskrf.ru/ratings/produkty-pitaniya/molochnye-produkty/smetana/
******  https://rskrf.ru/ratings/produkty-pitaniya/molochnye-produkty/tvorog/
******  https://rskrf.ru/rat

,point,category,sub_ctgr,dangrless,quality,rating,site
0,moloko-ekomilk,molochnie_produkti,moloko,95,79,79,https://roscontrol.com/
1,krasnaya-tsena-32-ultrapasterizovannoe,molochnie_produkti,moloko,83,76,71,https://roscontrol.com/
2,moloko-prostokvashino-3-2,molochnie_produkti,moloko,88,73,73,https://roscontrol.com/
3,selo-zelenoe-32-ultrapasterizovannoe,molochnie_produkti,moloko,91,84,77,https://roscontrol.com/
4,avida-32-pasterizovannoe,molochnie_produkti,moloko,92,79,76,https://roscontrol.com/
5,prostokvashino-otbornoe-ot-34-do-45-pasterizov...,molochnie_produkti,moloko,92,91,82,https://roscontrol.com/
6,ostankinskoe-32-ultrapasterizovannoe,molochnie_produkti,moloko,97,32,68,https://roscontrol.com/
7,sarafanovo-32-ultrapasterizovannoe,molochnie_produkti,moloko,97,49,71,https://roscontrol.com/
8,valio-32-ultrapasterizovannoe,molochnie_produkti,moloko,97,49,73,https://roscontrol.com/
9,avida-32-ultrapasterizovannoe,molochnie_produkti,moloko,97,15,63,https://roscontrol.com/
